In [8]:
pip install pefile yara-python

  Using cached pefile-2024.8.26-py3-none-any.whl.metadata (1.4 kB)
  Using cached yara_python-4.5.4-cp312-cp312-win_amd64.whl.metadata (3.0 kB)
Using cached pefile-2024.8.26-py3-none-any.whl (74 kB)
Using cached yara_python-4.5.4-cp312-cp312-win_amd64.whl (1.8 MB)
Note: you may need to restart the kernel to use updated packages.


# Hash Calculation (IOCs)

## Triage

Triage is an approach used to quickly assess and prioritise alerts and incidents based on risk and urgency. The triage process begins by analysts reviewing context and alerts, and determing what they mean, how serious they are, and who should handle them. This is important because it helps teams allocate limited time and resources on what matters most, for example a critical server under attack is more important than other systems. 

Triage reduces dwell time, streamlines response, and prevents minor issues from turning into full-blown incidents.

(https://www.legitsecurity.com/aspm-knowledge-base/triage-cyber-security)

## Comparing Hashes

Comparing the hash of a file against known hashes is the most efficient method analysts use to filter files. An analyst can compare the hash of the file against a database of known hashes (allowlist or a blocklist).

However, there is a problem. Changing even a single bit changes the entire hash, this is called the 'Avalance Effect'. This means an attacker can change a single bit to ensure that the hash does not match any of the known hashes, thus going undetected. The two ways to counter this are:
- **Fuzzy Hashing:** Compares how similar two files are, rather than if they are identical.
- **Import Hashing:** Hashes only the Import Address Table (the list of system functions the malware uses). Many malware variants use the exact same functions in the same order, even if the rest of the code changes.

## Hash Compare Simulation:

The following code simulates a basic hash comparison for the safe and benign file: `Procmon.exe`.

In [1]:
import hashlib

def compute_hash(path, algorithm):
    h = hashlib.new(algorithm)
    with open(path, "rb") as f:
        h.update(f.read())
    return h.hexdigest()

sample = r"Procmon.exe"

print("MD5: ", compute_hash(sample, "md5"))
print("SHA1: ", compute_hash(sample, "sha1"))
print("SHA256:", compute_hash(sample, "sha256"))

MD5:  c3e77b6959cc68baee9825c84dc41d9c
SHA1:  bc18a67ad4057dd36f896a4d411b8fc5b06e5b2f
SHA256: 3b7ea4318c3c1508701102cf966f650e04f28d29938f85d74ec0ec2528657b6e


### Demonstrating the avalanche effect:

The deletion of a single character from the `Procmon.exe` file completely changed the hash.

In [4]:
print("MD5: ", compute_hash('Procmon Edited.exe', "md5"))
print("SHA1: ", compute_hash('Procmon Edited.exe', "sha1"))
print("SHA256:", compute_hash('Procmon Edited.exe', "sha256"))

MD5:  fc6ec4cd1a85ec1079830c7d3cadcd83
SHA1:  681dad8d04ee758a24cd3dc9ee6b429b9473d637
SHA256: 4f7d5573f43e3fbfbd1b6ad5d9aef1afff7babf0130466535ec0e0b31aa2de58


# String Extraction

The following code uses the python regular expression module to excavate and reveal human-readable text. Analysts review these strings to identify:
- Hardcoded paths
- Registry keys
- Network infrastructure (domains, URLs, IPs)
- Encryption keys or markers
- Persistence mechanisms

Identifying these strings can give you insight on what the malware does, who may have written it and how to detect it.

In [6]:
import re

def extract_strings(path):
    with open(path, "rb") as f:
        data = f.read()
    pattern = rb"[ -~]{4,}"
    return re.findall(pattern, data)

strings = extract_strings(sample)

for s in strings:
    print(s.decode(errors="ignore"))


!This program cannot be run in DOS mode.
V*0T
0RichU
.text
`.rdata
@.data
.rsrc
@.reloc
hpqQ
h`EN
h|nN
h\nN
hlnN
=UUU
h_rM
hDLN
h`GO
hDLN
h|GO
hDLN
hl]O
hhzO
PhhzO
hxzO
PhxzO
h${O
Ph${O
h@{O
Ph@{O
hX{O
PhX{O
hHZN
PhHZN
ht{O
Pht{O
h<|O
Ph<|O
h\|O
Ph\|O
u`Vh
PQSVW
Y_^[
Y_^[
VWh$
u_j,
S$f;U
f9S&
Y_^[
Y_^[
uUj,
Y_^[
SVWP
98tj
Y_^[
h$~P
'u>h
hD<N
SVWh
Ph4=N
PhH=N
h(;N
hL;N
hh;N
h <N
h0<N
_^tD
hD<N
QSVW
u?h`<N
ht<N
SVWP
Y_^[
w!WPS
>_^[
>_^[
t7SWf
_[^]
SVWP
PWvA
PQVS
w|QR
w4QR
Y_^[
SVWP
^f;G
w(QR
Y_^[
SVWP
hhBN
gfff
w)QR
Y_^[
PhgBN
hLBN
wZQR
w(QW
~@ f
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
PQSVW
Y_^[
wEQR
w%QR
Y_^[
4SVW
Y_^[
?SQV
SQVR
w?VW
Y_^[
x\;X
SVW3
x{;X
PhpAN
PhpAN
PhpAN
Y_^[
SVWP
~#j,P
Y_^[
|oVh
Y_^[
w0VW
}'9p
Y_^[
Y_^[
rIWRQ
Y_^[
Y_^[
Y_^[
Y_^[
Y_^[
Y_^[
~@ f
DSVW
Y_^[
Y_^[
SVWP
xPQR
PQVS
wzQR
w2QR
Y_^[
 SVW
Y_^[
hl?N
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
SVWP
Y_^[
PRVW
w)QR
w)QR
@t(V
SVWP
Y_^[
 SVW
Y_^[
O8_^[
O _^[
;0v:f;
9~8s
9N8s
B,)8
B0)8

The file contains DLL names, such as 'WS2_32.dll' and 'VERSION.dll', and functions such as 'GetFileVersionInfoSizeW'. It also contains legitimate paths such as, 'http://www.microsoft.com/pkiops/Docs/Repository.htm0'. This shows that the file is benign.

## Packing

Packing is a technique used to compress and encrypt executable files to hide its content. In this process, funtions and strings get scrambled or encoded, making the string extaction code above redundant.

The packed program contains the compressed and encrypted data, as well as the 'stub'. When the program is run, the Unpacking Stub contains code that decrypts and decompresses the encrypted malware code in the system's memory.

Indicatiors of packed malware are:
- **High Entropy Values:** Normal code has structure, but packed code looks random.
- **Few Imports:** Unlike legitimate programs that contain many imports, packed malware contains minimal imports.
- **Section Names:** Suspicious defualt section names, such as '.UPX0'.

# PE Header Inspection Using `pefile`

In [9]:
import pefile

pe = pefile.PE(sample)

print("Entry Point:", hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint))
print("Image Base:", hex(pe.OPTIONAL_HEADER.ImageBase))

print("\nImported DLLs and functions:")
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(" ", entry.dll.decode())
    for imp in entry.imports[:5]:
        print(" -", imp.name.decode() if imp.name else "None")

Entry Point: 0xa7f70
Image Base: 0x400000

Imported DLLs and functions:
  WS2_32.dll
 - getsockname
 - listen
 - recv
 - closesocket
 - socket
  VERSION.dll
 - GetFileVersionInfoW
 - VerQueryValueW
 - GetFileVersionInfoSizeW
  COMCTL32.dll
 - ImageList_ReplaceIcon
 - ImageList_SetBkColor
 - ImageList_AddMasked
 - ImageList_BeginDrag
 - ImageList_EndDrag
  FLTLIB.DLL
 - FilterSendMessage
 - FilterGetMessage
 - FilterReplyMessage
 - FilterConnectCommunicationPort
  KERNEL32.dll
 - AcquireSRWLockExclusive
 - AcquireSRWLockShared
 - InitializeSRWLock
 - GetSystemInfo
 - VerSetConditionMask
  USER32.dll
 - LoadStringA
 - DrawEdge
 - GetMessageW
 - TranslateMessage
 - DispatchMessageW
  GDI32.dll
 - SaveDC
 - RestoreDC
 - SetBrushOrgEx
 - SetPixel
 - PatBlt
  COMDLG32.dll
 - ChooseColorW
 - GetOpenFileNameW
 - PrintDlgW
 - ChooseFontW
 - FindTextW
  ADVAPI32.dll
 - RegQueryValueExW
 - ConvertStringSidToSidW
 - ConvertSidToStringSidW
 - RegSetValueW
 - RegEnumKeyW
  SHELL32.dll
 - SHGetSpecia

Many legitimate Windows APIs, such as:
- `kernel32.dll`
- `user32.dll`
- `advapi32.dll`

# YARA Analysis

In [11]:
import yara

rule_source = """
rule ContainsHTTP {
    strings:
        $s = "http"
    condition:
        $s
}
"""

rules = yara.compile(source=rule_source)
matches = rules.match(sample)
print(matches)

[ContainsHTTP]


# Complete Static Triage Workflow

This is what the following code does:
1. Compute hashes
2. Extract readable strings
3. Enumerate imports
4. Identify potential IOCs
5. Apply a YARA rule

In [3]:
import hashlib, pefile, re, yara

sample = r"Procmon.exe"

def compute_hashes(path):
    algos = ["md5", "sha1", "sha256"]
    output = {}
    for a in algos:
        h = hashlib.new(a)
        with open(path, "rb") as f:
            h.update(f.read())
        output[a] = h.hexdigest()
    return output

def extract_strings(path):
    with open(path, "rb") as f:
        data = f.read()
    return re.findall(rb"[ -~]{4,}", data)

print("Hashes:", compute_hashes(sample))

print("\nStrings:")
print(extract_strings(sample)[:10])

print("\nImports:")
pe = pefile.PE(sample)
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print(entry.dll.decode())
    
print("\nIOCs:")
decoded = open(sample, "rb").read().decode(errors="ignore")
print("URLs:", re.findall(r"https?://[^\s\"']+", decoded))
print("IPs:", re.findall(r"\b\d{1,3}(?:\.\d{1,3}){3}\b", decoded))

print("\nYARA:")
rule = yara.compile(source="""
rule Simple {
    strings: $s = "http"
    condition: $s
}
""")
print(rule.match(sample))


Hashes: {'md5': 'c3e77b6959cc68baee9825c84dc41d9c', 'sha1': 'bc18a67ad4057dd36f896a4d411b8fc5b06e5b2f', 'sha256': '3b7ea4318c3c1508701102cf966f650e04f28d29938f85d74ec0ec2528657b6e'}

Strings:
[b'!This program cannot be run in DOS mode.', b'V*0T', b'0RichU', b'.text', b'`.rdata', b'@.data', b'.rsrc', b'@.reloc', b'hpqQ', b'h`EN']

Imports:
WS2_32.dll
VERSION.dll
COMCTL32.dll
FLTLIB.DLL
KERNEL32.dll
USER32.dll
GDI32.dll
COMDLG32.dll
ADVAPI32.dll
SHELL32.dll
ole32.dll
OLEAUT32.dll
SHLWAPI.dll
UxTheme.dll
dwmapi.dll
ntdll.dll

IOCs:
URLs: ['https://go.microsoft.com/fwlink/?LinkId=521839', 'https://go.microsoft.com/fwlink/?LinkId=521839', 'https://go.microsoft.com/fwlink/?LinkId=521839\\ul0\\cf0}}}}\\f0\\fs20', 'https://microsoft.com/exporting', 'https://microsoft.com/exporting}}}}\\f0\\fs19', 'http://www.microsoft.com/pkiops/crl/Microsoft%20Windows%20Third%20Party%20Component%20CA%202012.crl0\x06\x08+\x06\x01\x05\x05\x07\x01\x01\x04u0s0q\x06\x08+\x06\x01\x05\x05\x070\x02ehttp://www.microso